In [1]:
import pandas as pd
import numpy as np

In [2]:
# load data
df = pd.read_csv("combine.csv")
df.head()

,Date,Close,ma_5,ma_10,ma_20,ma_30,ma_60,ma_90,ma_180,ma_240,...,Nasdaq_diff,SP500_diff,INDU_diff,Oil_diff,Btc_diffpct,Gold_diffpct,Nasdaq_diffpct,SP500_diffpct,INDU_diffpct,Oil_diffpct
0,2012-01-03,5.32,4.975,4.975000,4.975000,4.975000,4.975000,4.975000,4.975000,4.975000,...,0.00,0.00,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2012-01-04,5.57,5.094,5.094000,5.094000,5.094000,5.094000,5.094000,5.094000,5.094000,...,-0.36,0.24,21.04,0.8793,0.046992,0.014944,-0.000136,0.000188,0.001697,0.019997
2,2012-01-05,6.65,5.508,5.353333,5.353333,5.353333,5.353333,5.353333,5.353333,5.353333,...,21.50,3.76,-2.72,0.0000,0.193896,-0.004148,0.008118,0.002944,-0.000219,0.000000
3,2012-01-06,6.90,5.888,5.574286,5.574286,5.574286,5.574286,5.574286,5.574286,5.574286,...,4.36,-3.25,-55.78,0.0000,0.037594,-0.009996,0.001633,-0.002537,-0.004493,0.000000
4,2012-01-09,6.90,6.850,5.972000,5.972000,5.972000,5.972000,5.972000,5.972000,5.972000,...,2.34,2.89,32.77,0.0000,0.000000,0.003576,0.000875,0.002262,0.002651,0.000000


In [3]:
# define to check if close price increased compare to the previous close
df["Trend"] = df.apply(lambda x: "up" if x["Btc_diff"] > 0 else ("down" if x["Btc_diff"] <0 else "nc"), axis=1)

In [4]:
df[["Date", "Trend"]]

,Date,Trend
0,2012-01-03,nc
1,2012-01-04,up
2,2012-01-05,up
3,2012-01-06,up
4,2012-01-09,nc
...,...,...
2318,2021-03-25,down
2319,2021-03-26,up
2320,2021-03-29,up
2321,2021-03-30,up


In [5]:
ind_train_start = 360
ind_test_start = int((len(df) - 360) * 0.8)

In [6]:
df.columns

Index(['Date', 'Close', 'ma_5', 'ma_10', 'ma_20', 'ma_30', 'ma_60', 'ma_90',
       'ma_180', 'ma_240', 'ma_360', 'Gold', 'Nasdaq_Comp', 'SP500', 'INDU',
       'Oil', 'Btc_diff', 'Gold_diff', 'Nasdaq_diff', 'SP500_diff',
       'INDU_diff', 'Oil_diff', 'Btc_diffpct', 'Gold_diffpct',
       'Nasdaq_diffpct', 'SP500_diffpct', 'INDU_diffpct', 'Oil_diffpct',
       'Trend'],
      dtype='object')

In [7]:
px = df[['ma_5', 'ma_10', 'ma_20', 'ma_30', 'ma_60', 'ma_90', 'ma_180', 'ma_240', 'ma_360', 'Gold_diff','Nasdaq_diff', 'SP500_diff', 'INDU_diff', 'Oil_diff']]
px.columns

Index(['ma_5', 'ma_10', 'ma_20', 'ma_30', 'ma_60', 'ma_90', 'ma_180', 'ma_240',
       'ma_360', 'Gold_diff', 'Nasdaq_diff', 'SP500_diff', 'INDU_diff',
       'Oil_diff'],
      dtype='object')

In [8]:
# change here
X = df[["Close"]]
# X = df[["ma_5", "ma_10", "ma_20", "ma_30", "ma_60", "ma_90", "ma_180", "ma_240", "ma_360"]]
#y = pd.get_dummies(df[["Trend"]], columns=["Trend"])
# predict Trend
#y = df["Trend"]

# predict Price
y = df[["Close"]]

In [9]:
X.shape

(2323, 1)

In [10]:
y.shape

(2323, 1)

In [11]:
X_train = X.iloc[ind_train_start:ind_test_start]
X_test = X.iloc[ind_test_start:]
X_test = X_test.iloc[:len(X_test)-1]
X_test.iloc[0]

Close    6919.07
Name: 1570, dtype: float64

In [12]:
print(X_train.shape, X_test.shape)
#print(X_train)

(1210, 1) (752, 1)


In [13]:
# for using other rows
#y_train = y.iloc[ind_train_start+1:ind_test_start+1]
#y_test = y.iloc[ind_test_start+1:]

# for using price
y_train = y.iloc[ind_train_start + 1:ind_test_start + 1]
y_test = y.iloc[ind_test_start+1:]

In [14]:
print(y_train.shape, y_test.shape)
#print(y_train)
y_test.iloc[0]

(1210, 1) (752, 1)


Close    6846.35
Name: 1571, dtype: float64

In [15]:
ind_test = ind_test_start + 3
ind_test2 = ind_test - ind_test_start
print(df[["Date", "Close", 'Gold', 'Nasdaq_Comp', 'SP500', 'INDU', 'Oil', "Btc_diff", "Trend"]].iloc[ind_test])
print(X_test.iloc[ind_test2])
print(y_test.iloc[ind_test2])

Date           2018-04-10
Close              6880.0
Gold                12.77
Nasdaq_Comp        7094.3
SP500             2656.87
INDU              24408.0
Oil               18.5393
Btc_diff          -289.99
Trend                down
Name: 1573, dtype: object
Close    6880.0
Name: 1573, dtype: float64
Close    6962.25
Name: 1574, dtype: float64


In [16]:
from sklearn.preprocessing import MinMaxScaler

In [17]:
X_scaler = MinMaxScaler().fit(X)
# for predict price
Y_scaler = MinMaxScaler().fit(y)

In [18]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
# for predict price
y_train_scaled = Y_scaler.transform(y_train)
y_test_scaled = Y_scaler.transform(y_test)

In [20]:
#y_train_scaled_shape = np.reshape(y_train_scaled, (y_train_scaled.shape[0]))
#print(y_train_scaled_shape.shape)

In [21]:
print(X_train_scaled)

[[0.00170753]
 [0.00175706]
 [0.00171463]
 ...
 [0.11725687]
 [0.12384775]
 [0.12253086]]


In [22]:
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model

SVC(kernel='linear')

In [23]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100],
              'gamma': [0.00001, 0.0005, 0.0001, 0.0005, 0.0001]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [24]:
grid.fit(X_train_scaled, y_train_scaled)

Fitting 5 folds for each of 55 candidates, totalling 275 fits
[CV 1/5] END ...........................C=0.001, gamma=1e-05; total time=   0.0s
[CV 2/5] END ...........................C=0.001, gamma=1e-05; total time=   0.0s
[CV 3/5] END ...........................C=0.001, gamma=1e-05; total time=   0.0s
[CV 4/5] END ...........................C=0.001, gamma=1e-05; total time=   0.0s
[CV 5/5] END ...........................C=0.001, gamma=1e-05; total time=   0.0s
[CV 1/5] END ..........................C=0.001, gamma=0.0005; total time=   0.0s
[CV 2/5] END ..........................C=0.001, gamma=0.0005; total time=   0.0s
[CV 3/5] END ..........................C=0.001, gamma=0.0005; total time=   0.0s
[CV 4/5] END ..........................C=0.001, gamma=0.0005; total time=   0.0s
[CV 5/5] END ..........................C=0.001, gamma=0.0005; total time=   0.0s
[CV 1/5] END ..........................C=0.001, gamma=0.0001; total time=   0.0s
[CV 2/5] END ..........................C=0.001,

C:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\sklearn\svm\_base.py", line 173, in fit
    y = self._validate_targets(y)
  File "C:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\sklearn\svm\_base.py", line 553, in _validate_targets
    check

[CV 2/5] END ............................C=0.1, gamma=0.0005; total time=   0.0s
[CV 3/5] END ............................C=0.1, gamma=0.0005; total time=   0.0s
[CV 4/5] END ............................C=0.1, gamma=0.0005; total time=   0.0s
[CV 5/5] END ............................C=0.1, gamma=0.0005; total time=   0.0s
[CV 1/5] END ............................C=0.1, gamma=0.0001; total time=   0.0s
[CV 2/5] END ............................C=0.1, gamma=0.0001; total time=   0.0s
[CV 3/5] END ............................C=0.1, gamma=0.0001; total time=   0.0s
[CV 4/5] END ............................C=0.1, gamma=0.0001; total time=   0.0s
[CV 5/5] END ............................C=0.1, gamma=0.0001; total time=   0.0s
[CV 1/5] END .............................C=0.5, gamma=1e-05; total time=   0.0s
[CV 2/5] END .............................C=0.5, gamma=1e-05; total time=   0.0s
[CV 3/5] END .............................C=0.5, gamma=1e-05; total time=   0.0s
[CV 4/5] END ...............

C:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\sklearn\svm\_base.py", line 173, in fit
    y = self._validate_targets(y)
  File "C:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\sklearn\svm\_base.py", line 553, in _validate_targets
    check

[CV 3/5] END .............................C=50, gamma=0.0005; total time=   0.0s
[CV 4/5] END .............................C=50, gamma=0.0005; total time=   0.0s
[CV 5/5] END .............................C=50, gamma=0.0005; total time=   0.0s
[CV 1/5] END .............................C=50, gamma=0.0001; total time=   0.0s
[CV 2/5] END .............................C=50, gamma=0.0001; total time=   0.0s
[CV 3/5] END .............................C=50, gamma=0.0001; total time=   0.0s
[CV 4/5] END .............................C=50, gamma=0.0001; total time=   0.0s
[CV 5/5] END .............................C=50, gamma=0.0001; total time=   0.0s
[CV 1/5] END .............................C=50, gamma=0.0005; total time=   0.0s
[CV 2/5] END .............................C=50, gamma=0.0005; total time=   0.0s
[CV 3/5] END .............................C=50, gamma=0.0005; total time=   0.0s
[CV 4/5] END .............................C=50, gamma=0.0005; total time=   0.0s
[CV 5/5] END ...............

C:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\sklearn\svm\_base.py", line 173, in fit
    y = self._validate_targets(y)
  File "C:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\sklearn\svm\_base.py", line 553, in _validate_targets
    check_classification_targets(y)
  File "C:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\multiclass.py", line 183, in check_classification_targets
    raise ValueError("Unknown label type: %r" % y_type)
ValueError: Unknown label type: 'continuous'

  warni

[CV 5/5] END ............................C=100, gamma=0.0005; total time=   0.0s
[CV 1/5] END ............................C=100, gamma=0.0001; total time=   0.0s
[CV 2/5] END ............................C=100, gamma=0.0001; total time=   0.0s
[CV 3/5] END ............................C=100, gamma=0.0001; total time=   0.0s
[CV 4/5] END ............................C=100, gamma=0.0001; total time=   0.0s
[CV 5/5] END ............................C=100, gamma=0.0001; total time=   0.0s
[CV 1/5] END ............................C=100, gamma=0.0005; total time=   0.0s
[CV 2/5] END ............................C=100, gamma=0.0005; total time=   0.0s
[CV 3/5] END ............................C=100, gamma=0.0005; total time=   0.0s
[CV 4/5] END ............................C=100, gamma=0.0005; total time=   0.0s
[CV 5/5] END ............................C=100, gamma=0.0005; total time=   0.0s
[CV 1/5] END ............................C=100, gamma=0.0001; total time=   0.0s
[CV 2/5] END ...............

ValueError: Unknown label type: 'continuous'

In [25]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 0.001, 'gamma': 1e-05}
nan
